Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:

import logging
from itertools import combinations

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from icecream import ic

Cost: $d + (d \cdot \alpha \cdot w)^\beta$ with $\alpha \ge 0$ and $\beta \ge 0$

## Genetico

In [53]:
from src.GA_solver import GA_Solver

In [ ]:

import importlib
from itertools import product
import Problem as ProblemModule
import src.GA_solver as GA_solver_module
import src.utils as utils_module
import s345905 as solution_module




importlib.reload(ProblemModule)
importlib.reload(GA_solver_module)
importlib.reload(utils_module)
importlib.reload(solution_module)

Problem = ProblemModule.Problem
GA_Solver = GA_solver_module.GA_Solver
compute_ga_params = utils_module.compute_ga_params
s345905_solution= solution_module.solution

cities_list = [50 ]
densities = [0.2, 0.8]
alphas = [0.2, 1, 2]
betas = [0.1, 0.2, 0.3]

results = []

for n_cities, density, alpha, beta in product(cities_list, densities, alphas, betas):
    p = Problem(n_cities, density=density, alpha=alpha, beta=beta)

    best_cost= s345905_solution(p)
    baseline_cost = p.baseline()
    print(
        f"{n_cities:4d} | d={density:.1f} | "
        f"a={alpha:.1f} | b={beta:.1f} | "
        f"rel={best_cost / baseline_cost * 100:.2f}% |"
        f"baseline={baseline_cost:.2f} | ga={best_cost:.2f} | "
    ) 
    results.append({
        "n_cities": n_cities,
        "density": density,
        "alpha": alpha,
        "beta": beta,
        "baseline_cost": baseline_cost,
        "ga_cost": best_cost,
        "relative_cost": best_cost / baseline_cost * 100
    })


  50 | d=0.2 | a=0.2 | b=0.1 | rel=53.60% |baseline=195.25 | ga=104.65 | 
  50 | d=0.2 | a=0.2 | b=0.2 | rel=62.90% |baseline=242.95 | ga=152.81 | 
  50 | d=0.2 | a=0.2 | b=0.3 | rel=71.48% |baseline=309.20 | ga=221.02 | 
  50 | d=0.2 | a=1.0 | b=0.1 | rel=54.16% |baseline=219.96 | ga=119.14 | 
  50 | d=0.2 | a=1.0 | b=0.2 | rel=64.12% |baseline=314.81 | ga=201.86 | 
  50 | d=0.2 | a=1.0 | b=0.3 | rel=73.25% |baseline=467.77 | ga=342.62 | 
  50 | d=0.2 | a=2.0 | b=0.1 | rel=56.36% |baseline=231.90 | ga=130.70 | 
  50 | d=0.2 | a=2.0 | b=0.2 | rel=63.05% |baseline=353.64 | ga=222.95 | 
  50 | d=0.2 | a=2.0 | b=0.3 | rel=73.81% |baseline=563.48 | ga=415.92 | 
  50 | d=0.8 | a=0.2 | b=0.1 | rel=68.98% |baseline=121.37 | ga=83.73 | 
  50 | d=0.8 | a=0.2 | b=0.2 | rel=74.50% |baseline=152.07 | ga=113.29 | 
  50 | d=0.8 | a=0.2 | b=0.3 | rel=79.40% |baseline=195.26 | ga=155.03 | 
  50 | d=0.8 | a=1.0 | b=0.1 | rel=71.70% |baseline=136.21 | ga=97.66 | 
  50 | d=0.8 | a=1.0 | b=0.2 | rel=77.40

In [ ]:

import s345905 as solution_module
importlib.reload(solution_module)
s345905_solution= solution_module.solution

logging.getLogger().setLevel(logging.WARNING)

test_cases = [
    # 1. Il caso "Standard" (Equilibrio)
    Problem(50, density=0.4, alpha=1.0, beta=1.0),
    
    # 2. Il "Corriere Espresso" (Peso quasi irrilevante, vince chi fa meno km)
    Problem(50, density=0.6, alpha=0.05, beta=1.0),
    
    # 3. Il "Trasporto Eccezionale" (Peso punitivo, vince chi torna spesso al deposito)
    Problem(50, density=0.5, alpha=2, beta=2),
    
    # 4. L'incubo non-lineare (Costo esplosivo col peso)
    Problem(50, density=0.3, alpha=0.8, beta=0.8),
    
    # 5. Grafo Labirintico (Pochi collegamenti, obbliga a percorsi tortuosi)
    Problem(50, density=0.05, alpha=0.5, beta=0.2),
    
    # 6. Il caso "Super-Alpha" (Quasi ogni città richiede un viaggio dedicato)
    Problem(50, density=0.8, alpha=10, beta=0.1)
]

for p in test_cases:
    print(f"\n\n--- PROBLEM: {p.graph.number_of_nodes()} cities, alpha {p.alpha}, beta {p.beta} ---")
    print(f"Baseline Cost: {p.baseline()}")

    best_cost=s345905_solution(p)
    print(f"\n\nGA Cost: {best_cost}")
    print (f"Relative cost: {best_cost/p.baseline()*100:.2f}")



--- PROBLEM: 50 cities, alpha 1.0, beta 1.0 ---
Baseline Cost: 9902.623370477148


GA Cost: 9900.394432149727
Relative cost: 99.98


--- PROBLEM: 50 cities, alpha 0.05, beta 1.0 ---
Baseline Cost: 498.14319191738497


GA Cost: 492.0014633160499
Relative cost: 98.77


--- PROBLEM: 50 cities, alpha 2, beta 2 ---
Baseline Cost: 7504461.099043543
